# 作业3：深度学习框架实践

本次作业将练习深度学习框架的使用，大部分内容用 PyTorch 实现。第1题利用卷积层和全连接层实现手写数字的识别，第2题利用 RNN 来实现英文名的自动生成，第3题是算法题，利用卷积运算实现任意大整数的乘法。

### 第1题

1. 目标：通过对 MNIST 数据进行训练，构建一个简单的图像分类模型，对图片中的数字进行识别。你将利用该模型对自己真实手写出的数字进行预测，观察模型效果。

2. 主要步骤：获取数据，创建模型结构，定义损失函数，编写训练循环，实施预测。

3. 获取数据。我们使用知名的 MNIST 数据集，它可以从 PyTorch 中利用工具函数下载得到。原始的 MNIST 数据训练集大小为60000，我们随机抽取其中的10000个观测进行简单的训练。以下函数会在当前目录建立一个名为 data 的文件夹，其中会包含下载得到的数据集。

**注意：请在任何程序的最开始加上随机数种子的设置。请保持这一习惯。**

In [56]:
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

np.random.seed(123)
torch.manual_seed(123)

mnist = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
loader = DataLoader(mnist, batch_size=10000, shuffle=True)

我们一次性取出随机抽取到的10000个观测，其中 x 是图片数据，y 是图片对应的数字。

In [57]:
x, y = next(iter(loader))

一个习惯性动作是查看数据的大小和维度。

In [58]:
print(x.shape)
print(y.shape)

torch.Size([10000, 1, 28, 28])
torch.Size([10000])


我们可以利用下面的函数展示图片的内容。如选择第一张图片，先将其转换成 Numpy 数组，再绘制图形：

In [59]:
import matplotlib.pyplot as plt

img = x[0].squeeze().cpu().numpy()
print(img.shape)
plt.imshow(img, cmap="gray")
plt.show()

(28, 28)


接下来请你选择5个你喜欢的数字（10000以下），然后取出对应位置的图片，并画出它们的内容。

In [60]:
# Plot five digits here
img1 = x[111].squeeze().cpu().numpy()
img2 = x[222].squeeze().cpu().numpy()
img3 = x[333].squeeze().cpu().numpy()
img4 = x[444].squeeze().cpu().numpy()
img5 = x[555].squeeze().cpu().numpy()
plt.figure()

plt.subplot(1,5,1)
plt.imshow(img1, cmap="gray")
plt.subplot(1,5,2)
plt.imshow(img2, cmap="gray")
plt.subplot(1,5,3)
plt.imshow(img3, cmap="gray")
plt.subplot(1,5,4)
plt.imshow(img4, cmap="gray")
plt.subplot(1,5,5)
plt.imshow(img5, cmap="gray")
plt.show()


4. 搭建模型。我们搭建一个类似于 LeNet-5 的网络，结构如下：

![](https://pic1.zhimg.com/80/v2-82eabb4c17e90d467197d013f7629f3c_720w.jpg)

我们需要创建2个卷积层、2个汇聚层（池化层）和2个全连接层，**暂时忽略所有的激活函数**。所有隐藏层的函数细节都可以在[官方文档](https://pytorch.org/docs/stable/nn.html)中按分类找到。每一个隐藏层本质上都是将一个数组变换成另一个数组的函数，因此为了确认编写的模型是正确的，可以先用一个小数据进行测试，观察输入和输出的维度。例如，我们先取出前10个观测，此时输入的维度是 `[10, 1, 28, 28]`：

In [61]:
smallx = x[0:10]
smally = y[0:10]
print(smallx.shape)
print(smally.shape)

torch.Size([10, 1, 28, 28])
torch.Size([10])


接下来创建第1个卷积层，并测试输出的维度。注意到我们可以直接将隐藏层当成一个函数来调用。

In [62]:
conv1 = torch.nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
res = conv1(smallx)
print(res.shape)

torch.Size([10, 20, 24, 24])


可以看到，输出的维度为 `[20, 24, 24]`（不包括第1位的数据批次维度），与之前图中的结果吻合。

接下来，请按照图中提示编写层对象 `pool1`、`conv2`、`pool2`、`fc1` 和 `fc2`，并顺次测试输入与输出的维度，使其与上图匹配。注意，在将一个大小为 `[10, 50, 4, 4]` 的数组（假设叫 `somearray`）传递给 `fc1` 之前，需要先将其变形为只有两个维度的数组，做法是 `somearray.view(-1, 50*4*4)`，其中 -1 表示该位置的大小不变。

```py
pool1 = ...
res = pool1(res)
print(res.shape)

conv2 = ...
res = conv2(res)
print(res.shape)

pool2 = ...
res = pool2(res)
print(res.shape)

fc1 = ...
res = fc1(res.view(-1, 50 * 4 * 4))
print(res.shape)

fc2 = ...
res = fc2(res)
print(res.shape)
```

In [63]:
pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
res = pool1(res)
print(res.shape)

torch.Size([10, 20, 12, 12])


In [64]:
conv2 = torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
res = conv2(res)
print(res.shape)

torch.Size([10, 50, 8, 8])


In [65]:
pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
res = pool2(res)
print(res.shape)

torch.Size([10, 50, 4, 4])


In [66]:
res = res.view(-1, 50*4*4)
print(res.shape)

torch.Size([10, 800])


In [67]:
fc1 = torch.nn.Linear(in_features=800,out_features=500,bias = True )
res = fc1(res)
print(res.shape)

torch.Size([10, 500])


In [68]:
res = torch.nn.functional.relu(res)
print(res.shape)

torch.Size([10, 500])


In [69]:
fc2 = torch.nn.Linear(in_features=500,out_features=10,bias = True )
res = fc2(res)
print(res.shape)

torch.Size([10, 10])


In [70]:
res = torch.nn.functional.softmax(res, dim = 0)
print(res.shape)

torch.Size([10, 10])


5. 创建模型类。在确保隐藏层维度都正确后，将所有的隐藏层封装到一个模型类中，其中模型结构在 `__init__()` 中定义，具体的计算过程在 `forward()` 中实现。此时需要加入激活函数。在本模型中，**请在 `conv1`、`conv2` 和 `fc1` 后加入 ReLU 激活函数，并在 `fc2` 后加入 Softmax 激活函数**。

```py
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ...
        self.pool1 = ...
        self.conv2 = ...
        self.pool2 = ...
        self.fc1 = ...
        self.fc2 = ...

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        ...
        return x
```

In [71]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(in_features=800,out_features=500,bias = True )
        self.fc2 = torch.nn.Linear(in_features=500,out_features=10,bias = True )

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(torch.nn.functional.relu(x))
        x = self.conv2(x)
        x = self.pool2(torch.nn.functional.relu(x))
        x = x.view(-1, 50*4*4)
        x = self.fc1(x)
        x = self.fc2(torch.nn.functional.relu(x))
        x = torch.nn.functional.softmax(x, dim = 1)
        return x

再次测试输入输出的维度是否正确。如果模型编写正确，输出的维度应该是 `[10, 10]`，且输出结果为0到1之间的概率值。

In [73]:
np.random.seed(1234)
torch.manual_seed(123)

model = MyModel()
pred = model(smallx)
print(pred.shape)
print(pred)

torch.Size([10, 10])
tensor([[0.0992, 0.0989, 0.0982, 0.1089, 0.0943, 0.0912, 0.0996, 0.1020, 0.1021,
         0.1056],
        [0.0997, 0.1004, 0.0978, 0.1085, 0.0938, 0.0907, 0.0982, 0.1045, 0.1020,
         0.1045],
        [0.0990, 0.0993, 0.0979, 0.1081, 0.0913, 0.0921, 0.0976, 0.1088, 0.1030,
         0.1031],
        [0.0955, 0.0996, 0.1010, 0.1081, 0.0931, 0.0913, 0.0988, 0.1044, 0.1031,
         0.1051],
        [0.0995, 0.0988, 0.0985, 0.1091, 0.0919, 0.0918, 0.0980, 0.1059, 0.1017,
         0.1049],
        [0.0977, 0.0994, 0.0996, 0.1073, 0.0924, 0.0922, 0.1018, 0.1019, 0.1040,
         0.1038],
        [0.0972, 0.1011, 0.1010, 0.1062, 0.0931, 0.0908, 0.1006, 0.1048, 0.1037,
         0.1016],
        [0.0996, 0.0984, 0.0987, 0.1068, 0.0904, 0.0900, 0.0992, 0.1076, 0.1020,
         0.1075],
        [0.0972, 0.0998, 0.1012, 0.1084, 0.0925, 0.0905, 0.1022, 0.1014, 0.1023,
         0.1045],
        [0.0979, 0.0998, 0.0985, 0.1080, 0.0959, 0.0910, 0.0979, 0.1049, 0.1027,
       

`pred` 的每一行加总为1，其中每一个元素代表对应类别的预测概率。

我们还可以直接打印模型对象，观察隐藏层的结构：

In [74]:
print(model)

MyModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


6. 损失函数。对于分类问题，损失函数通常选取为负对数似然函数。在 PyTorch 中，可以使用 `torch.nn.NLLLoss` 来完成计算。其用法是先定义一个损失函数对象，然后在预测值和真实标签上调用该函数对象。注意：损失函数对象的第一个参数是预测概率的**对数值**，第二个参数是真实的标签。[文档说明](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html)。

In [75]:
lossfn = torch.nn.NLLLoss()
lossfn(torch.log(pred), smally)

tensor(2.3231, grad_fn=<NllLossBackward>)

7. 利用课上介绍的循环模板和代码示例，对模型进行迭代训练。对于本数据，选取 mini-batch 大小为200，共遍历数据10遍，优化器选为 Adam，学习率为0.001。

In [76]:
from torch.autograd import Variable
from torch import optim
epochs = 10
batch_size = 200
lr = 0.001

loader = DataLoader(mnist, batch_size=batch_size, shuffle=True)
opimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)

for epoch in range(epochs):
    for step,(x,y) in enumerate(loader):
        model.train()
        b_x = Variable(x)
        b_y = Variable(y)
        
        out = model(b_x)
        loss = lossfn(torch.log(out), b_y.long())
        opimizer.zero_grad()
        loss.backward()
        opimizer.step()
        
        if step % 50 == 0:
            print('Epoch:',epoch,'|train loss:'+str(loss.item()))

Epoch: 0 |train loss:2.3035948276519775
Epoch: 0 |train loss:0.2413109391927719
Epoch: 0 |train loss:0.15266233682632446
Epoch: 0 |train loss:0.1261148899793625
Epoch: 0 |train loss:0.0986529141664505
Epoch: 0 |train loss:0.08441997319459915
Epoch: 1 |train loss:0.09275760501623154
Epoch: 1 |train loss:0.11651147902011871
Epoch: 1 |train loss:0.07878801226615906
Epoch: 1 |train loss:0.044405557215213776
Epoch: 1 |train loss:0.03689614310860634
Epoch: 1 |train loss:0.1033848226070404
Epoch: 2 |train loss:0.045710016041994095
Epoch: 2 |train loss:0.057826463133096695
Epoch: 2 |train loss:0.04169587045907974
Epoch: 2 |train loss:0.08028850704431534
Epoch: 2 |train loss:0.030156051740050316
Epoch: 2 |train loss:0.0606003999710083
Epoch: 3 |train loss:0.020447397604584694
Epoch: 3 |train loss:0.039062898606061935
Epoch: 3 |train loss:0.07325118780136108
Epoch: 3 |train loss:0.08342142403125763
Epoch: 3 |train loss:0.12424413859844208
Epoch: 3 |train loss:0.0678277313709259
Epoch: 4 |train l

为了验证模型的效果，我们对前10个观测（即之前生成的 `smallx` 和 `smally`）进行预测。

In [77]:
ypred = model(smallx)
print(np.round(ypred.detach().cpu().numpy(), 3))
print(smally)

[[0.    0.    0.    0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.    0.999 0.    0.    0.    0.    0.001]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.064 0.    0.936 0.    0.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.001 0.001 0.    0.    0.    0.998 0.    0.   ]
 [0.    0.993 0.    0.    0.001 0.    0.    0.006 0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.   ]]
tensor([7, 4, 5, 4, 0, 7, 1, 0, 7, 4])


如果模型搭建和训练都正常，那么每一行中概率最大的取值所在的位置应该正好对应真实的标签。

最后，我们用模型对一些真实的手写数字图片进行预测。请你利用绘图软件（如 Windows 自带的绘图，或 Photoshop 等）准备10张正方形黑色底色的图片，每张用鼠标绘制一个数字（请使用较粗的笔划），从0到9，然后以0.png，1.png等文件名存储下来，放到当前目录一个名为 digits 的文件夹中。以下是几个例子：
![](digits/sample0.png) ![](digits/sample5.png) ![](digits/sample8.png)

接下来利用 Pillow 软件包读取图片：

In [78]:
from PIL import Image

name=f"C:/Users/chenxuanzhu/Downloads/digits/sample0.png"
im = Image.open(name)


此时如果直接将其转为 Numpy 数组会得到三个通道：

In [79]:
im_arr = np.array(im)
print(im_arr.shape)

(261, 261, 3)


因此，我们先强制转换为灰度图片（单通道），再缩放至模型的图片大小 28 x 28：

In [80]:
im = im.convert("L")
im.thumbnail((28, 28))
im_arr = np.array(im)
print(im_arr.shape)
im

(28, 28)


为了传递给模型对象，还需要先将数值归一化到 [0,1] 区间，转换为 PyTorch 的 Tensor 类型，并增加一个批次和一个通道的维度：

In [81]:
test0 = torch.tensor(im_arr / 255.0, dtype=torch.float32).view(1, 1, 28, 28)
print(test0.shape)

torch.Size([1, 1, 28, 28])


最后对图片标签进行预测：

In [82]:
pred0 = model(test0)
print(np.round(pred0.detach().cpu().numpy(), 3))

[[0.436 0.    0.395 0.009 0.001 0.    0.006 0.    0.013 0.14 ]]


预测结果是否符合真实情形？请对你自己绘制出的10张图片进行类似的预测操作，并评价其效果。

In [84]:
from PIL import Image
for i in range(10):
    name=f"C:/Users/chenxuanzhu/Downloads/digits/{i}.png"
    im = Image.open(name)
    im_arr = np.array(im)
    im = im.convert("L")
    im.thumbnail((28, 28))
    im_arr = np.array(im)
    test0 = torch.tensor(im_arr / 255.0, dtype=torch.float32).view(1, 1, 28, 28)
    pred0 = model(test0)
    print(np.round(pred0.detach().cpu().numpy(), 3))

[[0.122 0.01  0.144 0.111 0.013 0.031 0.009 0.04  0.046 0.474]]
[[0.09  0.2   0.073 0.022 0.098 0.199 0.122 0.094 0.072 0.031]]
[[0.014 0.04  0.478 0.173 0.034 0.041 0.037 0.078 0.077 0.026]]
[[0.009 0.134 0.3   0.181 0.079 0.097 0.003 0.171 0.008 0.017]]
[[0.009 0.056 0.015 0.062 0.165 0.291 0.018 0.096 0.086 0.202]]
[[0.008 0.006 0.012 0.095 0.004 0.744 0.017 0.019 0.018 0.077]]
[[0.559 0.019 0.087 0.018 0.01  0.06  0.168 0.032 0.025 0.022]]
[[0.008 0.039 0.061 0.132 0.008 0.018 0.003 0.667 0.03  0.035]]
[[0.02  0.009 0.115 0.312 0.004 0.049 0.023 0.019 0.413 0.035]]
[[0.015 0.056 0.065 0.104 0.078 0.033 0.003 0.356 0.024 0.267]]


In [ ]:
看第1行0.474可知预测为0错误
看第2行0.2可知预测为1正确
看第3行0.478可知预测为2正确
看第4行0.181可知预测为3错误
看第5行0.291可知预测为4错误
看第6行0.744可知预测为5正确
看第7行0.599可知预测为6错误
看第8行0.667可知预测为7正确
看第9行0.413可知预测为8正确
看第10行0.267可知预测为9正确

### 第2题

1. 目标：通过对英文名数据进行训练，构建一个 RNN 模型，实现英文名的自动生成。

2. 主要步骤：获取和整理数据，对字符串进行 one-hot 编码，创建模型结构，定义损失函数，编写训练循环，最后生成人名字符串。

3. 获取和整理数据。数据文件已存为 `data/names.txt`，先将其读取为字符串列表：

In [1]:
import io
lines = io.open("C:/Users/chenxuanzhu/Downloads/data/names.txt").read().strip().split('\n')
print(len(lines))
print(lines[0:5])

3668
['Abbas', 'Abbey', 'Abbott', 'Abdi', 'Abel']


可以看出，共读取了3668个名字。为了简单起见，我们将所有的大写字母转换为小写。

In [2]:
names = [s.lower() for s in lines]
print(names[0:5])

['abbas', 'abbey', 'abbott', 'abdi', 'abel']


接下来我们需要构建一个字符的字典。对于英文名来说很简单，即26个字母。我们可以通过下面的代码直接得到。

In [3]:
import string
dict = string.ascii_lowercase
dict

'abcdefghijklmnopqrstuvwxyz'

4. 下面准备好 one-hot 编码所需的函数。编写函数 `char2index(char)`，将一个字母转换为其所在字典的位置。例如 `char2index("a")` 要返回0，`char2index("z")` 要返回25，等等。提示：使用字符串的 `.find()` 函数。

In [4]:
def char2index(char):
    return dict.find(char)

print(char2index("z") == 25)

True


编写 `char2tensor(char)` 函数，将一个字母转换为 one-hot 向量，即该向量中第 i 个元素为1，其余为0，其中 i 表示该字母在字典中的位置。

**注意，该向量的长度应为27，因为我们要预留终止符，用 `[0.0, 0.0, ..., 1.0]` 表示**。

`char2tensor("a")` 应返回 `torch.tensor([1.0, 0.0, ...])`，`char2tensor("z")` 应返回 `torch.tensor([0.0, ..., 1.0, 0.0])`。

In [5]:
import numpy as np
import torch
def char2tensor(char):
    tensor = torch.zeros(1, 27)
    tensor[0, char2index(char)] = 1
    return tensor

print(char2tensor("a"))
print(char2tensor("z"))
print(char2tensor("z").shape[0] == 27)


tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0.]])
False


5. 构建模型。我们使用最简单的 RNN 结构，即隐藏单元是输入和上一期隐藏单元的线性变换加上 Tanh 激活函数，输出单元是隐藏单元的线性变换加上 Softmax 激活函数。输出的结果代表下一个字符的概率分布。

```py
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = ...
        self.h2o = ...

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), dim=1)
        hidden = torch.tanh(self.i2h(combined))
        output = torch.nn.functional.softmax(self.h2o(hidden), dim=1)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
```

In [6]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = torch.nn.Linear(hidden_size, input_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), dim=1)
        hidden = torch.tanh(self.i2h(combined))
        output = torch.nn.functional.softmax(self.h2o(hidden), dim=1)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

我们做一个简单的测试。请在下面的代码中加入适当的语句，使得每次运行的结果不变。根据其输出结果，请问当前模型预测字符a的下一个字符是什么？为什么？

In [9]:
np.random.seed(1234)
torch.random.manual_seed(123)
device = torch.device("cpu")
rnn = RNN(input_size=27, hidden_size=10)
input = char2tensor("a")
hidden = rnn.init_hidden()
output, hidden = rnn(input.view(1, 27), hidden)
print(output)
torch.max(output,1)[1].data.squeeze()

tensor([[0.0290, 0.0327, 0.0320, 0.0354, 0.0432, 0.0395, 0.0417, 0.0371, 0.0409,
         0.0275, 0.0328, 0.0359, 0.0373, 0.0391, 0.0339, 0.0366, 0.0430, 0.0364,
         0.0472, 0.0435, 0.0376, 0.0327, 0.0418, 0.0370, 0.0279, 0.0426, 0.0357]],
       grad_fn=<SoftmaxBackward>)


tensor(18)

6. 接下来我们定义好损失函数。与第1题中类似，预测值是一个概率分布，而真实的标签是0到26中的一个整数，代表真实的下一个字符在字典中的位置。假设当前处理的名字为"abel"，那么字符a的输出结果对应的标签是什么？请完成下面的代码。

In [11]:
# Change "target" to a proper value
target = char2index("b")

lossfn = torch.nn.NLLLoss()
loss = lossfn(torch.log(output), torch.tensor([target]))
print(loss)

tensor(3.4199, grad_fn=<NllLossBackward>)


7. 明确单个字符的损失函数的计算方法后，请在下面计算出"abel"这个观测整体的损失函数值。

In [12]:
chac = list("abel")
loss = 0
for i in range(len(chac)):
    input = char2tensor(chac[i])
    hidden = rnn.init_hidden()
    output, hidden = rnn(input.view(1, 27), hidden)
    target = char2index(chac[i+1]) if i!= len(chac)-1 else  26
    loss += lossfn(torch.log(output), torch.tensor([target]))
loss = loss/len(chac)
print(loss)

tensor(3.2926, grad_fn=<DivBackward0>)


8. 将上述过程在数据上进行反复迭代，训练模型。

In [16]:
np.random.seed(1234)
torch.random.manual_seed(123)

n = len(names)
n_hidden = 16
n_input = 27
nepoch = 10
train_ind = np.arange(n)

rnn = RNN(n_input, n_hidden)
opt = torch.optim.Adam(rnn.parameters(), lr=0.0001)
train_ind = np.arange(n)
losses = []

lossfn = torch.nn.NLLLoss()

# Loop over epochs
for k in range(nepoch):
    # Shuffle the data
    np.random.shuffle(train_ind)
    # Loop over observations. Each observation is a name
    for i in range(n):
        name = names[train_ind[i]]
        nchar = len(name)
        # Loop over the characters in the name
        # Each input character has a target, which is the index of the next character in the dictionary
        # For the last character in the name, the target is the end-of-sequence symbol, which has index 26
        loss = 0.0
        hidden = rnn.init_hidden()
        for j in range(nchar):
            input = char2tensor(name[j])
            output, hidden = rnn(input.view(1, n_input), hidden)

            if j == nchar - 1:
                target = 26
            else:
                target = char2index(name[j + 1])

            loss = loss + lossfn(torch.log(output), torch.tensor([target]))
    
        loss = loss / nchar

        opt.zero_grad()
        loss.backward()
        opt.step()

        losses.append(loss.item())
        if i % 1000 == 0:
            print(f"epoch {k}, obs {i}, loss = {loss.item()}")

epoch 0, obs 0, loss = 3.2788641452789307
epoch 0, obs 1000, loss = 2.793740749359131
epoch 0, obs 2000, loss = 2.8677024841308594
epoch 0, obs 3000, loss = 2.556673765182495
epoch 1, obs 0, loss = 2.695556640625
epoch 1, obs 1000, loss = 2.5480618476867676
epoch 1, obs 2000, loss = 2.6293277740478516
epoch 1, obs 3000, loss = 2.432743549346924
epoch 2, obs 0, loss = 2.6687285900115967
epoch 2, obs 1000, loss = 2.425450325012207
epoch 2, obs 2000, loss = 2.685854911804199
epoch 2, obs 3000, loss = 2.7870194911956787
epoch 3, obs 0, loss = 2.547203540802002
epoch 3, obs 1000, loss = 2.3284201622009277
epoch 3, obs 2000, loss = 2.8587934970855713
epoch 3, obs 3000, loss = 3.0973093509674072
epoch 4, obs 0, loss = 2.2342605590820312
epoch 4, obs 1000, loss = 2.7695655822753906
epoch 4, obs 2000, loss = 2.6228115558624268
epoch 4, obs 3000, loss = 2.2542669773101807
epoch 5, obs 0, loss = 2.3726961612701416
epoch 5, obs 1000, loss = 2.792773962020874
epoch 5, obs 2000, loss = 2.62322187423

In [17]:
import matplotlib.pyplot as plt
plt.plot(losses)

9. 编写一个函数 `random_first_letter()`，它随机返回字典中的一个字符，我们将利用它来随机生成第一个字符。

In [18]:
def random_first_letter():
    # Implementation here
    return np.random.choice(list(dict), 1)[0]

print(random_first_letter())
print(random_first_letter())
print(random_first_letter())

#随机生成第一个字符
first_letter = random_first_letter()
char_ind = [char2index(first_letter)]
print(char_ind)


i
k
s
[9]


请简要说明如下的代码的含义（可以在代码中加入注释），然后利用它随机生成10个名字。评价生成的结果，并简要说明可以如何改进模型的效果？

In [19]:
def random_name(max_len=20): #生成一个最大长度为20名字的函数
    rnn.eval() 
    first_letter = random_first_letter()#随机选取第一个字母
    char_ind = [char2index(first_letter)]
    input = char2tensor(first_letter)#将第一个字母转化为tensor输入模型
    hidden = rnn.init_hidden()
    for i in range(max_len - 1):
        output, hidden = rnn(input.view(1, n_input), hidden) #输入网络
        ind = torch.argmax(output).item()
        if ind == 26:  #如果下一个为停止符号，就结束生成
            break
        char_ind.append(ind)
        input.zero_()
        input[0,ind] = 1.0
    return "".join([dict[i] for i in char_ind])

In [24]:
for i in range(10): 
    print(random_name())

garlen
tone
warlen
erer
warlen
urer
tone
qarlen
harlen
erer


In [25]:
##通过大部分名字感觉不太熟用，出现了多次arlen、erlen这种，可能是因为模型是通过一个字母预测下一个字母，可以通过改进模型使得用已有的全部名字去估计下一个字母。


### 第3题

利用卷积函数实现任意大整数的乘法。给定两个整数，如 183612 和 23333，用两个列表表达它们的序列：

In [47]:
n1 = [1, 8, 3, 6, 1, 2]
n2 = [2, 3, 3, 3, 3]

请编写一个函数 `integer_mult(n1, n2)`，返回 `n1 * n2` 对应的整数序列。注意不要直接调用乘法表达式（设想有两个非常大的整数，直接相乘可能会导致数值溢出）。

In [48]:
import numpy as np
def integer_mult(n1, n2):
    a = np.array(n1)
    b = np.array(n2)
    return(np.convolve(a,b))
    print(poly_mult(p, q))
print(integer_mult(n1, n2))

#res = integer_mult(n1, n2)
#print(res == [4, 2, 8, 4, 2, 1, 8, 7, 9, 6])

[ 2 19 33 48 56 58 36 27  9  6]


思路：

1. 先实现多项式的乘法。例如，给定 $p(x)=1+2x+x^4$ 和 $q(x)=x+3x^2+5x^3$，计算 $r(x)=p(x)q(x)$。我们将 $p(x)$ 编码为 `p = [1, 2, 0, 0, 1]`，$q(x)$ 编码为 `q = [0, 1, 3, 5]`，请编写函数 `poly_mult(p, q)`，使得 `poly_mult(p, q) == [0, 1, 5, 11, 10, 1, 3, 5]`。

2. 对于任意的一个整数，将其看成是某个多项式在 $x=10$ 处的取值，如 $123 = p_1(10)$，$p_1(x)=3+2x+x^2$，$5310 = p_2(10)$，$p_2(x)=x+3x^2+5x^3$，注意需要适当将序列反序。因此，要计算 $123\times 5310$，相当于计算 $r(10)$ 的值，但为了避免直接进行乘法运算（防止溢出），可以先计算 $r(x)$ 的表达式（等价于其系数向量），然后建立起 $r(x)$ 的系数与 $r(10)$ 之间的联系（见如下第3点）。

3. 如果一个多项式 $r(x)$ 所有的系数都是0到9之间的整数，那么 $r(x)$ 和 $r(10)$ 的关系非常直接，比如若 $r(x)=1+2x+5x^2+3x^3$，则 $r(10)=3521$。但如果有系数超过10，就需要考虑进位的影响，比如 $r(x)=1+11x+2x^2$，$r(10)=311$。此时可以从 $r(x)$ 的第一项开始逐项进位，构造一个新的多项式 $r'(x)=1+x+3x^2$，满足 $r'(10)=r(10)$，且 $r'(x)$ 所有的系数都不超过10。 

In [41]:
def new_poly(ans):
    for i in range(len(ans)-1):
        out = ans[i]//10
        stay = ans[i]%10
        ans[i+1] += out
        ans[i] = stay
    b = ans[-1]
    ans.pop(-1)
    while(b//10 >0):
        ans.append(b%10)
        b //= 10
    ans.append(b)
    return ans

4. 综合利用以上信息，完成本题的算法编写。并测试 23742389754298365 * 809723950 的结果。

In [42]:
l1 = [int(x) for x in '23742389754298365']
l2 = [int(x) for x in '809723950']
def integer_mult(n1, n2):
    ans = poly_mult(n1, n2)
    return new_poly(ans)

res = list(reversed(integer_mult(list(reversed(l1)), list(reversed(l2)))))
print(res)

[1.0, 9.0, 2.0, 2.0, 4.0, 7.0, 8.0, 1.0, 6.0, 1.0, 4.0, 2.0, 9.0, 0.0, 0.0, 0.0, 1.0, 5.0, 8.0, 6.0, 3.0, 4.0, 1.0, 7.0, 5.0, 0.0]
